## [step1] 필요 패키지 import

In [1]:
import tensorflow as tf
import numpy as np

## [step2] tf.data.Dataset.from_tensor_slices

In [2]:
data = [1, 2, 3, 4, 5, 6]

ds_tensors = tf.data.Dataset.from_tensor_slices(data)
ds_tensors = ds_tensors.map(tf.square).shuffle(2,seed=1337).batch(2) 

print(ds_tensors)

<BatchDataset shapes: (None,), types: tf.int32>


In [3]:
# 알아서 배치 단위로 나눠짐 + tf.square 한번에 적용됨
print('ds_tensors 요소:')
for x in ds_tensors:
    print(x)

ds_tensors 요소:
tf.Tensor([1 9], shape=(2,), dtype=int32)
tf.Tensor([ 4 25], shape=(2,), dtype=int32)
tf.Tensor([16 36], shape=(2,), dtype=int32)


In [8]:
data = np.array([1,2,3])
label = tf.constant([1,0,1])
ds_tensors = tf.data.Dataset.from_tensor_slices((data,label))
print(ds_tensors)

<TensorSliceDataset shapes: ((), ()), types: (tf.int32, tf.int32)>


In [10]:
print('ds_tensors 요소:')
for datas,label in ds_tensors:
    print(datas,label)

ds_tensors 요소:
tf.Tensor(1, shape=(), dtype=int32) tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32) tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32) tf.Tensor(1, shape=(), dtype=int32)


## [step3] tf.data.Dataset.from_generator

In [14]:
## 예시 generator만들기
## 매 iteration마다 size 5의 데이터셋 생성 
def generator():
    i = 0
    while True:
        i += 1
        yield i,np.random.normal(size=(5,))

In [15]:
for i,data in generator():
    if i==4:
        break
    else:
        print(data)

[-0.21446993 -0.30019629  0.27384571  0.18406654 -1.2370853 ]
[-0.57960261  0.6260057   0.14134328 -0.04743251 -0.77035223]
[-0.44897354 -0.23336105 -0.13403598  0.37024056  0.18414725]


In [16]:
dataset=tf.data.Dataset.from_generator(generator,
                                    output_types=(tf.int32,tf.float32)
                                    ,output_shapes=((),(None,))) ## output shape,type 명시해 줘야함

dataset=dataset.shuffle(2).batch(2) # batch, map,shuffle 동일하게 적용가능
count = 0
for i,data in dataset:
    if count == 2:
        break
    count += 1    
    print(i,data)                               

tf.Tensor([2 3], shape=(2,), dtype=int32) tf.Tensor(
[[-1.4653198   0.0450115   1.3250165   1.1400386   1.4883908 ]
 [-0.9571643   1.4044256  -0.9744767   0.5907449   0.11195796]], shape=(2, 5), dtype=float32)
tf.Tensor([4 1], shape=(2,), dtype=int32) tf.Tensor(
[[ 0.22942866 -0.36993876  0.6292869  -1.0530798  -0.13331817]
 [-1.5747483  -0.09895548  0.4214286   0.19795027 -0.6644921 ]], shape=(2, 5), dtype=float32)


In [17]:
def generator():
    i = 0
    while True:
        i += 1
        size = np.random.randint(0,10)
        yield i,np.random.normal(size=(size,))
for i,data in generator():
    if i==4:
        break
    else:
        print("size:",len(data),data)

size: 6 [ 0.6996667  -2.37343738 -0.51912281 -0.85889919 -0.0917889  -1.66402974]
size: 4 [0.27394016 1.74123595 0.14564884 0.40191271]
size: 1 [0.21308418]


In [18]:
## padded_batch를 활용한 제로패딩

dataset=tf.data.Dataset.from_generator(generator,
                                    output_types=(tf.int32,tf.float32)
                                    ,output_shapes=((),(None,)))
dataset = dataset.padded_batch(4,padded_shapes=([],[None]))
next(iter(dataset))                            

(<tf.Tensor: id=154, shape=(4,), dtype=int32, numpy=array([1, 2, 3, 4])>,
 <tf.Tensor: id=155, shape=(4, 7), dtype=float32, numpy=
 array([[-0.5757371 ,  0.13540089,  0.00243155, -1.1434231 ,  1.0883269 ,
          0.4876261 ,  0.        ],
        [-0.30536646,  0.6795862 , -0.42502084,  0.46519905,  0.718154  ,
         -0.9253375 ,  0.41803196],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 1.1269614 ,  0.9358637 , -0.47013015,  0.7901065 ,  0.        ,
          0.        ,  0.        ]], dtype=float32)>)